In [ ]:
"""
Steps:
1. get the total counts images (ps excluded) of 0.7-1.2 keV
1.1 convert 0.5-4keV flux to 0.7-1.2 keV flux in webpimms

2. extract the annu if the region is above certain counts
3. save the region as det and sky coords

"""

In [ ]:
# load the centers from the regions
from glob import glob
import numpy as np
import pandas as pd


regpath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/fk5_regs'
regdirs = glob(f'{regpath}/SDSSTG*')
output_dict = {}
output_dict['ID'] = []
output_dict['RA'] = []
output_dict['DEC'] = []
output_dict['01R500'] = []
for  dir in regdirs:
    if glob(f'{dir}/R500-01_fk5.reg'):
        with open(f'{dir}/R500-01_fk5.reg') as f:
            lines = f.readlines()
            for line in lines:
                if 'circle' in line:
                    output_dict['ID'].append(dir.split('/')[-1])
                    output_dict['RA'].append(line.split(',')[0].split('(')[-1])
                    output_dict['DEC'].append(line.split(',')[1])
                    output_dict['01R500'].append(line.split(',')[-1].split('\'')[0])
                
print(output_dict)
df = pd.DataFrame(output_dict)
df.to_csv(f'{regpath}/all_centers.csv')


In [ ]:
""" 
Procedures:
1. Get images of 0.5-7 keV
1.1 load the centers
2. Create multiple circle regions centered by
3. count the sum counts in every circle
3. Select the radius of SNR>50 or total cts > 2500 (3000?)

"""
from glob import glob
import numpy as np


# get images of 0.7-7 keV
rootpath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits_fan'
datdirs = glob(f'{rootpath}/SDSSTG*')
regf = pd.read_csv(f'{dir}/../../alldata/fk5_regs/all_centers.csv')
for dir in datdirs:
    ID = dir.split('/')[-1]
    msk = regf['ID'] == ID
    ra = regf['RA'][msk]
    dec = regf['DEC'][msk]
    
    DATFILE = f'comb-fovimsky-500-7000_masked.fits'
    
    with open(f'{dir}/{DATFILE}') as f:
        
    


2.1 cts>3000

In [ ]:
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u
from glob import glob
import numpy as np
from astropy.io import fits
from photutils import CircularAperture, aperture_photometry
import pandas as pd

# get images of 0.7-7 keV
rootpath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits_fan'
datdirs = glob(f'{rootpath}/SDSSTG*')
savepath = f'{rootpath}/../../fk5_regs'
regf = pd.read_csv(f'{savepath}/all_centers.csv')

output_dict_name = f'{savepath}/cts3000_reg.csv'
output_dict = {}
output_dict['ID'] = []
output_dict['RA'] = []
output_dict['DEC'] = []
output_dict['r'] = []


for dir in datdirs:
    ID = dir.split('/')[-1]
    msk = regf['ID'] == ID
    ra = regf['RA'][msk]
    dec = regf['DEC'][msk]
    
    print(ra, dec)
    DATFILE = f'comb-fovimsky-500-7000_masked.fits'
    if glob(f'{dir}/{DATFILE}'):
        with fits.open(f'{dir}/{DATFILE}') as hdul:
            data = hdul[0].data
            header = hdul[0].header

            # Create a WCS object to convert RA and Dec to pixel coordinates
            w = wcs.WCS(header)

            # Create a list to store the selected regions
            selected_regions = []

            # Iterate over different radii in arcminutes
            for radius_arcmin in np.arange(0.5, 10, 0.1):
                # Convert radius to pixel coordinates
                radius_pixel = np.round((radius_arcmin / 60.0) / w.wcs.cdelt[1])

                # Convert RA and Dec to pixel coordinates
                sky_coords = SkyCoord(ra, dec, unit='deg')
                pixel_coords = sky_coords.to_pixel(w)

                # Create CircularAperture objects for each region
                apertures = CircularAperture(list(zip(pixel_coords[0], pixel_coords[1])), r=radius_pixel)

                # Perform aperture photometry
                phot_table = aperture_photometry(data, apertures)

                # Assuming 'aperture_sum' is the column containing the sum counts in each circle
                sum_counts = phot_table['aperture_sum']
                
                # Select regions based on total counts > 3000
                if sum_counts > 3000:
                    # Add the selected regions to the list
                    output_dict['ID'].append(ID)
                    output_dict['RA'].append(np.round(regf.loc[regf['ID'] == ID, 'RA'].values[0], decimals = 2))
                    output_dict['DEC'].append(np.round(regf.loc[regf['ID'] == ID, 'DEC'].values[0], decimals = 2))
                    output_dict['r'].append(np.round(radius_arcmin, decimals = 1))
                    break

df = pd.DataFrame(output_dict)
df.to_csv(output_dict_name)

2.2 SNR>50

In [ ]:
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u
from glob import glob
import numpy as np
from astropy.io import fits
from photutils import CircularAperture, aperture_photometry
import pandas as pd

# get images of 0.7-7 keV
rootpath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits_fan'
datdirs = glob(f'{rootpath}/SDSSTG*')
savepath = f'{rootpath}/../../fk5_regs'
regf = pd.read_csv(f'{savepath}/all_centers.csv')

output_dict_name = f'{savepath}/SNR50_reg.csv'
output_dict = {}
output_dict['ID'] = []
output_dict['RA'] = []
output_dict['DEC'] = []
output_dict['r'] = []


for dir in datdirs:
    ID = dir.split('/')[-1]
    msk = regf['ID'] == ID
    ra = regf['RA'][msk]
    dec = regf['DEC'][msk]
    
    print(ra, dec)
    DATFILE = f'comb-fovimsky-500-7000_masked.fits'
    if glob(f'{dir}/{DATFILE}'):
        with fits.open(f'{dir}/{DATFILE}') as hdul:
            data = hdul[0].data
            header = hdul[0].header

            # Create a WCS object to convert RA and Dec to pixel coordinates
            w = wcs.WCS(header)

            # Create a list to store the selected regions
            selected_regions = []

            # Iterate over different radii in arcminutes
            for radius_arcmin in np.arange(0.1, 10, 0.1):
                # Convert radius to pixel coordinates
                radius_pixel = np.round((radius_arcmin / 60.0) / w.wcs.cdelt[1])

                # Convert RA and Dec to pixel coordinates
                sky_coords = SkyCoord(ra, dec, unit='deg')
                pixel_coords = sky_coords.to_pixel(w)

                # Create CircularAperture objects for each region
                apertures = CircularAperture(list(zip(pixel_coords[0], pixel_coords[1])), r=radius_pixel)

                # Perform aperture photometry
                phot_table = aperture_photometry(data, apertures)

                # Assuming 'aperture_sum' is the column containing the sum counts in each circle
                sum_counts = phot_table['aperture_sum']
                src_area = 4 * np.pi * radius_arcmin**2
                annu_ctr = sum_counts / src_area # cts/arcmin2
                # get bkg ctr
                r1 = np.round((9.89513 / 60.0) / w.wcs.cdelt[1])
                r2 = np.round((12.1242 / 60.0) / w.wcs.cdelt[1])
                ap1 = CircularAperture(list(zip(pixel_coords[0], pixel_coords[1])), r=r1)
                ap2 = CircularAperture(list(zip(pixel_coords[0], pixel_coords[1])), r=r2)
                sc1 = aperture_photometry(data, ap1)
                sc2 = aperture_photometry(data, ap2)
                bkg_counts1 = sc1['aperture_sum'].data[0]
                bkg_counts2 = sc2['aperture_sum'].data[0]
                bkg_area = np.pi * 4 * (12.1242**2 - 9.89513**2)
                bkg_ctr = (bkg_counts2 - bkg_counts1) / bkg_area # cts/arcmin2


                # Select regions based on total counts > 3000
                if annu_ctr > 50 * bkg_ctr:
                    # Add the selected regions to the list
                    output_dict['ID'].append(ID)
                    output_dict['RA'].append(np.round(regf.loc[regf['ID'] == ID, 'RA'].values[0], decimals = 2))
                    output_dict['DEC'].append(np.round(regf.loc[regf['ID'] == ID, 'DEC'].values[0], decimals = 2))
                    output_dict['r'].append(np.round(radius_arcmin, decimals = 1))
                    break

df = pd.DataFrame(output_dict)
df.to_csv(output_dict_name)

2.3 plot contours

In [98]:
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u
from glob import glob
import numpy as np
from astropy.io import fits
from photutils import CircularAperture, aperture_photometry
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.ndimage import gaussian_filter
import os

# get images of 0.7-7 keV
rootpath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits_ctr_500-7000'
datdirs = glob(f'{rootpath}/SDSSTG*')
savepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits_ctr_500-7000_contour'
os.makedirs(savepath, exist_ok=True)
regf = pd.read_csv(f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/fk5_regs/all_centers.csv')

output_dict_name = f'{savepath}/SNR50_reg.csv'
output_dict = {}
output_dict['ID'] = []
output_dict['RA'] = []
output_dict['DEC'] = []
output_dict['r'] = []


for dir in datdirs:
    ID = dir.split('/')[-1]
    msk = regf['ID'] == ID
    ra = regf['RA'][msk]
    dec = regf['DEC'][msk]
    
    DATFILE = f'comb-ctr-500-7000_masked_trimmed.fits'
    if glob(f'{dir}/{DATFILE}'):
        with fits.open(f'{dir}/{DATFILE}') as hdul:
            id = dir.split('/')[-1]
            data = hdul[0].data
            header = hdul[0].header

            # Create a WCS object to convert RA and Dec to pixel coordinates
            w = wcs.WCS(header)

            # Create a list to store the selected regions
            selected_regions = []

           # Convert RA and Dec to pixel coordinates
            sky_coords = SkyCoord(ra, dec, unit='deg')
            pixel_coords = sky_coords.to_pixel(w)

            # get bkg ctr
            bkg_r1 = 6
            bkg_r2 = 12
            r1 = np.round((bkg_r1 / 60.0) / w.wcs.cdelt[1])
            r2 = np.round((bkg_r2 / 60.0) / w.wcs.cdelt[1])
            ap1 = CircularAperture(list(zip(pixel_coords[0], pixel_coords[1])), r=r1)
            ap2 = CircularAperture(list(zip(pixel_coords[0], pixel_coords[1])), r=r2)
            sc1 = aperture_photometry(data, ap1)
            sc2 = aperture_photometry(data, ap2)
            bkg_counts1 = sc1['aperture_sum'].data[0]
            bkg_counts2 = sc2['aperture_sum'].data[0]
            bkg_area = np.pi * (bkg_r2**2 - bkg_r1**2) * 3600/2.5**2 # pixel
            bkg_ctr = (bkg_counts2 - bkg_counts1) / bkg_area # cts/s/pixel
            # Plot contours based on X-ray surface brightness
            fig, ax = plt.subplots()
            smoothed_data = gaussian_filter(data, sigma=25)
            im = ax.imshow(data, cmap='plasma', vmin = data.min()/2, vmax = 0.1*data.max(), origin = 'lower')
            cir1 = plt.Circle((pixel_coords[0], pixel_coords[1]), r2, color = 'cyan', fill = False)
            cir2 = plt.Circle((pixel_coords[0], pixel_coords[1]), r1, color = 'cyan', fill = False)
            ax.add_patch(cir1)
            ax.add_patch(cir2)
            # Add colorbar for X-ray surface brightness
            cs = ax.contour(smoothed_data, levels = (bkg_ctr, 1.2 * bkg_ctr, 1.5 * bkg_ctr, 3 * bkg_ctr), colors = ['w', 'magenta', 'r', 'yellow'], linewidths=1, origin='lower')
            plt.savefig(f'{savepath}/{id}_comb-fovimsky-500-7000_masked_contour.png')
            plt.close()


/var/folders/sy/vxhnb6x11wz6mkt7x_sw_y140000gn/T/ipykernel_5821/3420915476.py:7: DeprecationWarning: `photutils.CircularAperture` is a deprecated alias for `photutils.aperture.CircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAperture` to silence this warning.
  from photutils import CircularAperture, aperture_photometry
/var/folders/sy/vxhnb6x11wz6mkt7x_sw_y140000gn/T/ipykernel_5821/3420915476.py:7: DeprecationWarning: `photutils.aperture_photometry` is a deprecated alias for `photutils.aperture.aperture_photometry` and will be removed in the future. Instead, please use `from photutils.aperture import aperture_photometry` to silence this warning.
  from photutils import CircularAperture, aperture_photometry
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-END to 59975.505799 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-END to 59810.

In [79]:
print(bkg_ctr)

0.000809476343527634


In [68]:
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u
from glob import glob
import numpy as np
from astropy.io import fits
from photutils import CircularAperture, aperture_photometry
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.ndimage import gaussian_filter
import os

# get images of 0.7-7 keV
rootpath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits_ctr_500-7000'
datdirs = glob(f'{rootpath}/SDSSTG*')
savepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits_ctr_500-7000_contour'
os.makedirs(savepath, exist_ok=True)
# Loop over all regions and plot contours based on X-ray surface brightness
for dir in datdirs:
    DATFILE = f'comb-ctr-500-7000_masked_trimmed.fits'
    if glob(f'{dir}/{DATFILE}'):
        id = dir.split('/')[-1]
        with fits.open(f'{dir}/{DATFILE}') as hdul:
            data = hdul[0].data
            header = hdul[0].header


            # Plot contours based on X-ray surface brightness
            fig, ax = plt.subplots()
            smoothed_data = gaussian_filter(data, sigma=20)
            im = ax.imshow(data, cmap='plasma', vmin = data.min()/2, vmax = 0.1*data.max(), origin = 'lower')

            # Add colorbar for X-ray surface brightness
            cs = ax.contour(smoothed_data, levels = np.logspace(-5.7, np.log10(data.max()), 10), colors = 'w', linewidths=1, origin='lower')
            plt.savefig(f'{savepath}/{id}_comb-fovimsky-500-7000_masked_contour.png')
            plt.close()

/var/folders/sy/vxhnb6x11wz6mkt7x_sw_y140000gn/T/ipykernel_5821/2869274860.py:7: DeprecationWarning: `photutils.CircularAperture` is a deprecated alias for `photutils.aperture.CircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAperture` to silence this warning.
  from photutils import CircularAperture, aperture_photometry
/var/folders/sy/vxhnb6x11wz6mkt7x_sw_y140000gn/T/ipykernel_5821/2869274860.py:7: DeprecationWarning: `photutils.aperture_photometry` is a deprecated alias for `photutils.aperture.aperture_photometry` and will be removed in the future. Instead, please use `from photutils.aperture import aperture_photometry` to silence this warning.
  from photutils import CircularAperture, aperture_photometry


In [63]:
np.log10(smoothed_data.min()), np.log10(smoothed_data.max())

/var/folders/sy/vxhnb6x11wz6mkt7x_sw_y140000gn/T/ipykernel_5821/905136847.py:1: RuntimeWarning: divide by zero encountered in log10
  np.log10(smoothed_data.min()), np.log10(smoothed_data.max())


(-inf, -4.76391)

In [ ]:
print()

3 make plots

In [ ]:
# read 01r500c
import matplotlib.pyplot as plt
df1 = pd.read_csv(f'{savepath}/all_centers.csv')
# read r that cts>3000
df2 = pd.read_csv(f'{savepath}/cts3000_reg.csv')

plt.scatter(df1['01R500'], df2['r'])
plt.xlabel('0.1R500c')
plt.ylabel('cts>3000 [<r]')
plt.plot(np.arange(0,4), np.arange(0,4))
plt.axis('equal')
plt.xlim(0,4)
plt.ylim(0,4)

In [ ]:
print(selected_regions)